In [1]:
import tensorflow as tf
import os

In [2]:
data_dir = '../../../Other/datasets/cats_vs_dogs'

train_cats_dir = data_dir + '/train/cats/'
train_dogs_dir = data_dir + '/train/dogs/'
train_tfrecord_file = data_dir + '/train/train.tfrecords'

#### 写入

In [3]:
train_cat_filenames = [train_cats_dir + filename for filename in os.listdir(train_cats_dir)]
train_dog_filenames = [train_dogs_dir + filename for filename in os.listdir(train_dogs_dir)]
train_filenames = train_cat_filenames + train_dog_filenames

train_labels = [0] * len(train_cat_filenames) + [1] * len(train_dog_filenames)

In [4]:
with tf.io.TFRecordWriter(train_tfrecord_file) as writer:
    for filename, label in zip(train_filenames, train_labels):
        # 1,数据img,label
        image = open(filename, 'rb').read()

        # 2,Features字典
        feature = {
            'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),  # image的数据类型为bytes
            'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))  # label的数据类型为int
        }

        # 3,Features字典转换为Example对象并序列化
        example = tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()

        # 4,写入到TFRecord文件
        writer.write(example)

#### 加载

In [5]:
# 1. 读取TFRecore文件
train_dataset = tf.data.TFRecordDataset(train_tfrecord_file)

# 2,定义Feature结构,告诉解码器每个Feature的类型是什么
feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64),
}


def _parse_example(example_string):
    # 3,将TFRecord文件中的每一个序列化的tf.train.Example解码
    feature_dict = tf.io.parse_single_example(example_string, feature_description)

    feature_dict['image'] = tf.io.decode_jpeg(feature_dict['image'])  # 解码JPEG图片
    feature_dict['image'] = tf.image.resize(feature_dict['image'], [256, 256]) / 255.0
    return feature_dict['image'], feature_dict['label']

In [6]:
# 数据处理
train_dataset = train_dataset.map(_parse_example)
train_dataset = train_dataset.shuffle(buffer_size=23000)
train_dataset = train_dataset.batch(16)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
for i in train_dataset.take(1):
    print(i)

(<tf.Tensor: shape=(16, 256, 256, 3), dtype=float32, numpy=
array([[[[0.30958182, 0.30958182, 0.31742495],
         [0.29961655, 0.29961655, 0.30745968],
         [0.28392646, 0.28392646, 0.2917696 ],
         ...,
         [0.09019608, 0.10588235, 0.10196079],
         [0.09019608, 0.10588235, 0.10196079],
         [0.09019608, 0.10588235, 0.10196079]],

        [[0.35026807, 0.35026807, 0.3581112 ],
         [0.34155944, 0.34155944, 0.34940258],
         [0.3292916 , 0.3292916 , 0.33713475],
         ...,
         [0.09019608, 0.10588235, 0.10196079],
         [0.09019608, 0.10588235, 0.10196079],
         [0.09019608, 0.10588235, 0.10196079]],

        [[0.3725064 , 0.3725064 , 0.38034955],
         [0.3672258 , 0.3672258 , 0.37506893],
         [0.36068618, 0.36068618, 0.36852932],
         ...,
         [0.09019608, 0.10588235, 0.10196079],
         [0.09019608, 0.10588235, 0.10196079],
         [0.09019608, 0.10588235, 0.10196079]],

        ...,

        [[0.513084  , 0.45764592